In [1]:
#import libraries
import requests
import pandas as pd
import time

In [2]:
#Keys
API_KEY = "XXXXXX"
CHANNEL_ID = "XXXXXX"

In [10]:
def get_video_details(video_id):

    #collecting view, like, dislike, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    favorite_count = response_video_stats['items'][0]['statistics']['favoriteCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, favorite_count, comment_count

In [11]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, favorite_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,"favorite_count":favorite_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [12]:
#main

#build our dataframe
df2 = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","favorite_count","comment_count"]) 

df2 = get_videos(df2)

In [13]:
df2

,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
0,gtHI672Tlbw,DoorDash Medium Level SQL Interview Question,2022-09-08,59,4,0,1
1,DmUR2QSNUq8,DoorDash Data Science SQL Interview Question W...,2022-09-07,720,38,0,5
2,OLG6_EHMhFk,The One Thing That Keeps Data Scientists Up-to...,2022-08-31,461,37,0,4
3,0EoaJE3ePcE,Use STRING_AGG Function to Solve SQL Questions,2022-08-25,1363,67,0,3
4,JDOAcKTy9Mk,Top 5 Data Science Project Ideas in 2022 to St...,2022-08-18,1982,114,0,12
...,...,...,...,...,...,...,...
90,UX4_IgagL9I,How to Use Google Colaboratory | Import a CSV ...,2020-05-02,2285,12,0,2
91,tDdo3FiWpgE,Interview Questions for SQL Joins and Subqueries,2020-02-01,1723,26,0,0
92,wW827gqxlRY,SQL Job Interview Mistakes #2,2019-09-15,1127,21,0,0
93,xbc2GpGUXwc,SQL Job Interview Mistakes #1,2019-08-03,8025,74,0,1


In [15]:
df2.to_csv('youtube_vids_data_pull.csv')